# Constraint Reification -- an example

Cesira is a nice elder living in Romagna. Her nephew Carlo studies at the University, and every time Carlo achieves a high grade in some exam, Cesira is proud and makes a present of 50€ to Carlo.

Let's say that Carlo achieves a mark `v`, the gain `g` of Carlo is defined as:
- if `v>=28`, then `g==50`
- if `v<28`, then `g==0`

We can also switch viewpoint, and observe that:
- if Carlo has a gain `g==50`, then he has achieved a mark `v>=28`
- if Carlo has a gain `g==0`, then he has achieves a mark `v<28`

We can take one viewpoint or the other, but in both cases there is a variable whose constraints depend on the fact that another constraint is currently satisfied or not. E.g., the constraint `g==50` should be enforced only if the constraint `v>=28` is true.

How to model such situation?

## Reification

The solution stnads in introducing a third variable, that captures the state True/False of one constraint or the other. Through such vraible, it will be possible to specify that the constraint shoud be forced or not.

We would like to rewrite the cases above in the following way:  
`b == (v>=28)`  
where the intended meaning would be that `b` is `True` if and only if the constraint `(v>=28)` (double implication).  
Then we would like to express also that:  
`b == (g==50)`  
where the intended meaning is that the constraint about `g` is enforced if `b` is `True`, and viceversa.

In [1]:
from ortools.sat.python import cp_model

class VarArraySolutionPrinter(cp_model.CpSolverSolutionCallback):
    """Print intermediate solutions."""

    def __init__(self, variables):
        cp_model.CpSolverSolutionCallback.__init__(self)
        self.__variables = variables
        self.__solution_count = 0

    def on_solution_callback(self):
        self.__solution_count += 1
        for v in self.__variables:
            print('%s=%i' % (v, self.Value(v)), end=' ')
        print()

    def solution_count(self):
        return self.__solution_count

In [2]:
from ortools.sat.python import cp_model

model = cp_model.CpModel()

# The two initial variables
v = model.NewIntVar(18,30, 'v')
g = model.NewIntVarFromDomain(cp_model.Domain.FromIntervals([[0],[50]]), 'g')



# Compute the solution
solver = cp_model.CpSolver()
solution_printer = VarArraySolutionPrinter([v,g])

solver.parameters.enumerate_all_solutions = True
status = solver.Solve(model, solution_printer)
print('Solutions found:', solution_printer.solution_count())

v=18 g=0 
v=18 g=50 
v=19 g=50 
v=20 g=50 
v=21 g=50 
v=22 g=50 
v=23 g=50 
v=24 g=50 
v=25 g=50 
v=26 g=50 
v=27 g=50 
v=28 g=50 
v=29 g=50 
v=30 g=50 
v=30 g=0 
v=29 g=0 
v=28 g=0 
v=27 g=0 
v=26 g=0 
v=25 g=0 
v=24 g=0 
v=23 g=0 
v=22 g=0 
v=21 g=0 
v=20 g=0 
v=19 g=0 
Solutions found: 26


If you compute the solution above, you will see that a link is missing: the `g` is not related at all with the characteristics of the values of `v`, and viceversa.

To overcome this problem, in the next cell we will introduce a reification variable, and create the link betwween the two.

In [3]:
from ortools.sat.python import cp_model

model = cp_model.CpModel()

# The two initial variables
v = model.NewIntVar(18,30, 'v')
g = model.NewIntVarFromDomain(cp_model.Domain.FromIntervals([[0],[50]]), 'g')

# A boolean variable b
b = model.NewBoolVar('b')

# The reification link
model.Add(v>=28).OnlyEnforceIf(b)
model.Add(v<28).OnlyEnforceIf(b.Not())
model.Add(g==50).OnlyEnforceIf(b)
model.Add(g==0). OnlyEnforceIf(b.Not())

# Compute the solution
solver = cp_model.CpSolver()
solution_printer = VarArraySolutionPrinter([v,g])

solver.parameters.enumerate_all_solutions = True
status = solver.Solve(model, solution_printer)
print('Solutions found:', solution_printer.solution_count())

v=18 g=0 
v=19 g=0 
v=20 g=0 
v=21 g=0 
v=22 g=0 
v=23 g=0 
v=24 g=0 
v=25 g=0 
v=26 g=0 
v=27 g=0 
v=28 g=50 
v=29 g=50 
v=30 g=50 
Solutions found: 13
